# Assignment 4: Self-Attention for Vision

For this assignment, we're going to implement self-attention blocks in a convolutional neural network for CIFAR-10 Classification.

# Part I. Preparation

First, we load the CIFAR-10 dataset. This might take a couple minutes the first time you do it, but the files should stay cached after that.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [2]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./data/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./data/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./data/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


You have an option to **use GPU by setting the flag to True below**. It is not necessary to use GPU for this assignment. Note that if your computer does not have CUDA enabled, `torch.cuda.is_available()` will return False and this notebook will fallback to CPU mode.

The global variables `dtype` and `device` will control the data types throughout this assignment. 

In [3]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


## Flatten Function

In [4]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

def test_flatten():
    x = torch.arange(12).view(2, 1, 3, 2)
    print('Before flattening: ', x)
    print('After flattening: ', flatten(x))

test_flatten()

Before flattening:  tensor([[[[ 0,  1],
          [ 2,  3],
          [ 4,  5]]],


        [[[ 6,  7],
          [ 8,  9],
          [10, 11]]]])
After flattening:  tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])


### Check Accuracy Function


In [5]:
import torch.nn.functional as F  # useful stateless functions
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        return 100 * acc

### Training Loop

In [6]:
def train(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    acc_max = 0
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Epoch %d, Iteration %d, loss = %.4f' % (e, t, loss.item()))
                acc = check_accuracy(loader_val, model)
                if acc >= acc_max:
                    acc_max = acc
                print()
    print("Maximum accuracy attained: ", acc_max)

In [7]:
# We need to wrap `flatten` function in a module in order to stack it
# in nn.Sequential
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

## Vanilla CNN; No Attention
We implement the vanilla architecture for you here. Do not modify the architecture. You will use the same architecture in the following parts. Do not modify the hyper-parameters.

In [17]:
channel_1 = 64
channel_2 = 32
learning_rate = 1e-3
num_classes = 10

model = nn.Sequential(
    nn.Conv2d(3, channel_1, 3, padding=1, stride=1),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.ReLU(),
    Flatten(),
    nn.Linear(channel_2*32*32, num_classes),
)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)


train(model, optimizer, epochs=1)

Epoch 0, Iteration 0, loss = 2.3019
Checking accuracy on validation set
Got 132 / 1000 correct (13.20)

Epoch 0, Iteration 100, loss = 1.9547
Checking accuracy on validation set
Got 451 / 1000 correct (45.10)

Epoch 0, Iteration 200, loss = 1.6075
Checking accuracy on validation set
Got 485 / 1000 correct (48.50)

Epoch 0, Iteration 300, loss = 1.3779
Checking accuracy on validation set
Got 522 / 1000 correct (52.20)

Epoch 0, Iteration 400, loss = 1.3859
Checking accuracy on validation set
Got 524 / 1000 correct (52.40)

Epoch 0, Iteration 500, loss = 1.3557
Checking accuracy on validation set
Got 529 / 1000 correct (52.90)

Epoch 0, Iteration 600, loss = 1.2682
Checking accuracy on validation set
Got 546 / 1000 correct (54.60)

Epoch 0, Iteration 700, loss = 1.1904
Checking accuracy on validation set
Got 560 / 1000 correct (56.00)

Maximum accuracy attained:  56.00000000000001


## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.
You should be able to see atleast 55% accuracy

In [18]:
vanillaModel = model
check_accuracy(loader_test, vanillaModel)


Checking accuracy on test set
Got 5722 / 10000 correct (57.22)


57.220000000000006

## Part II Self-Attention

In the next section, you will implement an Attention layer which you will then use within a convnet architecture defined above for cifar 10 classification task.

A self-attention layer is formulated as following:

Input: $X$ of shape $(H\times W, C)$

Query, key, value linear transforms are $W_Q$, $W_K$, $W_V$, of shape $(C, C)$. We implement these linear transforms as 1x1 convolutional layers of the same dimensions.

$XW_Q$, $XW_K$, $XW_V$, represent the output volumes when input X is passed through the transforms.


Self-Attention is given by the formula: $Attention(X) = X + Softmax(\frac{XW_Q(XW_K)^\top}{\sqrt{C}})XW_V$

### Inline Question 1: Self-Attention is equivalent to which of the following: (5 points)
1. K-means clustering <br />
2. Non-local means <br />
3. Residual Block <br />
4. Gaussian Blurring <br />

Your Answer:Non-local means

Reason: Non-local means is used in image processing for denoising. It replaces pixel values with the average in the local nieghbourhood. This is similar to Self-Attention which basically finds representation of a sequences based on the similarity/importance of sequence.

### Here you implement the Attention module, and run it in the next section (40 points)

In [9]:
# Initialize the attention module as a nn.Module subclass
class Attention(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        
        # TODO: Implement the Key, Query and Value linear transforms as 1x1 convolutional layers
        # Hint: channel size remains constant throughout
        self.conv_query = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.conv_key = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.conv_value = nn.Conv2d(in_channels, in_channels, kernel_size=1)

    def forward(self, x):
        N, C, H, W = x.shape
        
        # TODO: Pass the input through conv_query, reshape the output volume to (N, C, H*W)
        q = self.conv_query(x).view(N, C, H * W)
        # TODO: Pass the input through conv_key, reshape the output volume to (N, C, H*W)
        k = self.conv_key(x).view(N, C, H * W)
        # TODO: Pass the input through conv_value, reshape the output volume to (N, C, H*W)
        v = self.conv_value(x).view(N, C, H * W)
        # TODO: Implement the above formula for attention using q, k, v, C
        # NOTE: The X in the formula is already added for you in the return line
        attention_scores = torch.matmul(q.transpose(1, 2), k) / (C ** 0.5)
        attention_weights = torch.softmax(attention_scores, dim=-1)
        weighted_values = torch.matmul(attention_weights, v.transpose(1, 2))
        attention = weighted_values.transpose(1, 2).view(N, C, H, W)
        # Reshape the output to (N, C, H, W) before adding to the input volume
        attention = attention.reshape(N, C, H, W)
        return x + attention

## Single Attention Block: Early attention; After the first conv layer. (10 points)

In [11]:
channel_1 = 64
channel_2 = 32
learning_rate = 1e-3
num_classes = 10

# TODO: Use the above Attention module after the first Convolutional layer.
# Essentially the architecture should be [Conv->Relu->Attention->Relu->Conv->Relu->Linear]

model = nn.Sequential(
    nn.Conv2d(3, channel_1, 3, padding=1, stride=1),
    nn.ReLU(),
    Attention(channel_1),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.ReLU(),
    Flatten(),
    nn.Linear(channel_2*32*32, num_classes),
)

# model = None

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 2.3030
Checking accuracy on validation set
Got 117 / 1000 correct (11.70)

Epoch 0, Iteration 100, loss = 1.8734
Checking accuracy on validation set
Got 355 / 1000 correct (35.50)

Epoch 0, Iteration 200, loss = 1.8629
Checking accuracy on validation set
Got 386 / 1000 correct (38.60)

Epoch 0, Iteration 300, loss = 1.6716
Checking accuracy on validation set
Got 442 / 1000 correct (44.20)

Epoch 0, Iteration 400, loss = 1.5178
Checking accuracy on validation set
Got 472 / 1000 correct (47.20)

Epoch 0, Iteration 500, loss = 1.4998
Checking accuracy on validation set
Got 462 / 1000 correct (46.20)

Epoch 0, Iteration 600, loss = 1.3074
Checking accuracy on validation set
Got 487 / 1000 correct (48.70)

Epoch 0, Iteration 700, loss = 1.6935
Checking accuracy on validation set
Got 516 / 1000 correct (51.60)

Epoch 1, Iteration 0, loss = 1.0585
Checking accuracy on validation set
Got 529 / 1000 correct (52.90)

Epoch 1, Iteration 100, loss = 1.2043
Checking acc

## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.
You should see improvement of about 2-3% over the vanilla convnet model. * Use this part to tune your Attention module and then move on to the next parts. *

In [12]:
earlyAttention = model
check_accuracy(loader_test, earlyAttention)

Checking accuracy on test set
Got 5939 / 10000 correct (59.39)


59.39

## Single Attention Block: Late attention; After the second conv layer. (10 points)

In [15]:
channel_1 = 64
channel_2 = 32
learning_rate = 1e-3

# TODO: Use the above Attention module after the Second Convolutional layer.
# Essentially the architecture should be [Conv->Relu->Conv->Relu->Attention->Relu->Linear]

model = nn.Sequential(
    nn.Conv2d(3, channel_1, 3, padding=1, stride=1),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.ReLU(),
    Attention(channel_2),
    nn.ReLU(),
    Flatten(),
    nn.Linear(channel_2*32*32, num_classes),
)

# model = None

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 2.2865
Checking accuracy on validation set
Got 137 / 1000 correct (13.70)

Epoch 0, Iteration 100, loss = 1.6135
Checking accuracy on validation set
Got 468 / 1000 correct (46.80)

Epoch 0, Iteration 200, loss = 1.3887
Checking accuracy on validation set
Got 480 / 1000 correct (48.00)

Epoch 0, Iteration 300, loss = 1.1509
Checking accuracy on validation set
Got 498 / 1000 correct (49.80)

Epoch 0, Iteration 400, loss = 1.2787
Checking accuracy on validation set
Got 546 / 1000 correct (54.60)

Epoch 0, Iteration 500, loss = 1.1799
Checking accuracy on validation set
Got 558 / 1000 correct (55.80)

Epoch 0, Iteration 600, loss = 1.0566
Checking accuracy on validation set
Got 538 / 1000 correct (53.80)

Epoch 0, Iteration 700, loss = 1.0984
Checking accuracy on validation set
Got 574 / 1000 correct (57.40)

Epoch 1, Iteration 0, loss = 1.1564
Checking accuracy on validation set
Got 584 / 1000 correct (58.40)

Epoch 1, Iteration 100, loss = 1.1347
Checking acc

## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.

In [16]:
lateAttention = model
check_accuracy(loader_test, lateAttention)

Checking accuracy on test set
Got 6001 / 10000 correct (60.01)


60.01

### Inline Question 2: Provide one example each of usage of self-attention and attention in computer vision. Explain the difference between the two. (5 points)


Your Answer: self-Attention is used in Vision Transformers and Attention is used in Squeeze-and-Excitation Networks (SENet)

Self-Attention is designed to relate all parts of input data to each other. Attention focus on enhancing specific features of the data already processed by the network layers. Self-Attention helps in learning relationships between distant parts of image and help understanding complex scenes. Attention helps in focusing more locally and contextually based on the features extracted.

## Double Attention Blocks: After conv layers 1 and 2 (10 points)

In [21]:
channel_1 = 64
channel_2 = 32
learning_rate = 1e-3

# TODO: Use the above Attention module after the Second Convolutional layer.
# Essentially the architecture should be [Conv->Relu->Attention->Relu->Conv->Relu->Attention->Relu->Linear]

model = nn.Sequential(
    nn.Conv2d(3, channel_1, 3, padding=1, stride=1),
    nn.ReLU(),
    Attention(channel_1),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.ReLU(),
    Attention(channel_2),
    nn.ReLU(),
    Flatten(),
    nn.Linear(channel_2*32*32, num_classes),
)

# model = None

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 2.3143
Checking accuracy on validation set
Got 139 / 1000 correct (13.90)

Epoch 0, Iteration 100, loss = 1.7504
Checking accuracy on validation set
Got 386 / 1000 correct (38.60)

Epoch 0, Iteration 200, loss = 1.7770
Checking accuracy on validation set
Got 455 / 1000 correct (45.50)

Epoch 0, Iteration 300, loss = 1.4278
Checking accuracy on validation set
Got 466 / 1000 correct (46.60)

Epoch 0, Iteration 400, loss = 1.3953
Checking accuracy on validation set
Got 500 / 1000 correct (50.00)

Epoch 0, Iteration 500, loss = 1.5924
Checking accuracy on validation set
Got 481 / 1000 correct (48.10)

Epoch 0, Iteration 600, loss = 1.4789
Checking accuracy on validation set
Got 509 / 1000 correct (50.90)

Epoch 0, Iteration 700, loss = 1.4244
Checking accuracy on validation set
Got 518 / 1000 correct (51.80)

Epoch 1, Iteration 0, loss = 1.0272
Checking accuracy on validation set
Got 524 / 1000 correct (52.40)

Epoch 1, Iteration 100, loss = 1.5940
Checking acc

## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.

In [22]:
vanillaModel = model
check_accuracy(loader_test, vanillaModel)

Checking accuracy on test set
Got 6143 / 10000 correct (61.43)


61.42999999999999

## Resnet with Attention 

Now we will experiment with applying attention within the Resnet10 architecture that we implemented in Homework 2. Please note that for a deeper model such as Resnet we do not expect significant improvements in performance with Attention

## Vanilla Resnet, No Attention

The architecture for Resnet is given below, please train it and evaluate it on the test set.

In [24]:
import torch
import torch.nn as nn

class ResNet(nn.Module):

    def __init__(self, block, layers, img_channels=3, num_classes=100, batchnorm=False):
        super(ResNet, self).__init__() #layers = [1, 1, 1, 1] 
        self.in_channels = 64
        self.conv1 = nn.Conv2d(img_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.batchnorm = batchnorm
        self.layer1 = self.make_layer(block, layers[0], out_channels=64, stride=1, batchnorm=batchnorm)
        self.layer2 = self.make_layer(block, layers[1], out_channels=128, stride=1, batchnorm=batchnorm)
        self.layer3 = self.make_layer(block, layers[2], out_channels=256, stride=1, batchnorm=batchnorm)
        self.layer4 = self.make_layer(block, layers[3], out_channels=512, stride=2, batchnorm=batchnorm)

        self.averagepool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    
    def forward(self, x):

        x = self.conv1(x)
        if self.batchnorm:
            x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x) 
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.averagepool(x)
        x = x.reshape(x.shape[0], -1)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x


        

    def make_layer(self, block, num_blocks, out_channels, stride, batchnorm=False):
        downsampler = None
        layers = []
        if stride != 1 or self.in_channels != out_channels:
            downsampler = nn.Sequential(nn.Conv2d(self.in_channels, out_channels, kernel_size = 1, stride = stride), nn.BatchNorm2d(out_channels))

        layers.append(block(self.in_channels, out_channels, downsampler, stride, batchnorm=batchnorm))

        self.in_channels = out_channels

        for i in range(num_blocks - 1):
            layers.append(block(self.in_channels, out_channels))

        
        return nn.Sequential(*layers)
        
class block(nn.Module):

    def __init__(self, in_channels, out_channels, downsampler = None, stride = 1, batchnorm=False):
        
        super(block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 2)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = stride)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsampler = downsampler
        self.relu = nn.ReLU()
        self.batchnorm = batchnorm

    
    def forward(self, x):

        residual = x
        x = self.conv1(x)
        if self.batchnorm:
            x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        if self.batchnorm:
            x = self.bn2(x)
        x = self.relu(x)
        
        if self.downsampler:
            residual = self.downsampler(residual)

        return self.relu(residual + x)
    


def ResNet10(num_classes = 100, batchnorm= False):

    return ResNet(block, [1, 1, 1, 1], num_classes=num_classes, batchnorm=batchnorm)

## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.

In [26]:
learning_rate = 1e-3

model = ResNet10()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

vanillaResnet = model
check_accuracy(loader_test, vanillaResnet)

Epoch 0, Iteration 0, loss = 4.7022
Checking accuracy on validation set
Got 105 / 1000 correct (10.50)

Epoch 0, Iteration 100, loss = 1.5347
Checking accuracy on validation set
Got 429 / 1000 correct (42.90)

Epoch 0, Iteration 200, loss = 1.4403
Checking accuracy on validation set
Got 403 / 1000 correct (40.30)

Epoch 0, Iteration 300, loss = 1.3525
Checking accuracy on validation set
Got 474 / 1000 correct (47.40)

Epoch 0, Iteration 400, loss = 1.2986
Checking accuracy on validation set
Got 453 / 1000 correct (45.30)

Epoch 0, Iteration 500, loss = 1.2994
Checking accuracy on validation set
Got 534 / 1000 correct (53.40)

Epoch 0, Iteration 600, loss = 1.2611
Checking accuracy on validation set
Got 519 / 1000 correct (51.90)

Epoch 0, Iteration 700, loss = 1.0467
Checking accuracy on validation set
Got 558 / 1000 correct (55.80)

Epoch 1, Iteration 0, loss = 1.1865
Checking accuracy on validation set
Got 532 / 1000 correct (53.20)

Epoch 1, Iteration 100, loss = 1.0678
Checking acc

75.31

## Resnet with Attention (5 points)

In [29]:
## Resnet with Attention

learning_rate = 1e-3

# TODO: Use the above Attention module after the 2nd resnet block i.e. after self.layer2.

class ResNetWithAttention(nn.Module):

    def __init__(self, block, layers, img_channels=3, num_classes=100, batchnorm=False):
        super(ResNetWithAttention, self).__init__() #layers = [1, 1, 1, 1] 
        self.in_channels = 64
        self.conv1 = nn.Conv2d(img_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.batchnorm = batchnorm
        self.layer1 = self.make_layer(block, layers[0], out_channels=64, stride=1, batchnorm=batchnorm)
        self.layer2 = self.make_layer(block, layers[1], out_channels=128, stride=1, batchnorm=batchnorm)
        self.attention =  Attention(128)
        self.layer3 = self.make_layer(block, layers[2], out_channels=256, stride=1, batchnorm=batchnorm)
        self.layer4 = self.make_layer(block, layers[3], out_channels=512, stride=2, batchnorm=batchnorm)

        self.averagepool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    
    def forward(self, x):

        x = self.conv1(x)
        if self.batchnorm:
            x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x) 
        x = self.layer2(x)
        x = self.attention(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.averagepool(x)
        x = x.reshape(x.shape[0], -1)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x


        

    def make_layer(self, block, num_blocks, out_channels, stride, batchnorm=False):
        downsampler = None
        layers = []
        if stride != 1 or self.in_channels != out_channels:
            downsampler = nn.Sequential(nn.Conv2d(self.in_channels, out_channels, kernel_size = 1, stride = stride), nn.BatchNorm2d(out_channels))

        layers.append(block(self.in_channels, out_channels, downsampler, stride, batchnorm=batchnorm))

        self.in_channels = out_channels

        for i in range(num_blocks - 1):
            layers.append(block(self.in_channels, out_channels))

        
        return nn.Sequential(*layers)

def ResNet10WithAttention(num_classes = 100, batchnorm= False):

    return ResNetWithAttention(block, [1, 1, 1, 1], num_classes=num_classes, batchnorm=batchnorm)
    
model = ResNet10WithAttention()

# model = None

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 4.6136
Checking accuracy on validation set
Got 119 / 1000 correct (11.90)

Epoch 0, Iteration 100, loss = 1.4601
Checking accuracy on validation set
Got 411 / 1000 correct (41.10)

Epoch 0, Iteration 200, loss = 1.3689
Checking accuracy on validation set
Got 489 / 1000 correct (48.90)

Epoch 0, Iteration 300, loss = 1.1874
Checking accuracy on validation set
Got 435 / 1000 correct (43.50)

Epoch 0, Iteration 400, loss = 1.1685
Checking accuracy on validation set
Got 527 / 1000 correct (52.70)

Epoch 0, Iteration 500, loss = 1.0757
Checking accuracy on validation set
Got 513 / 1000 correct (51.30)

Epoch 0, Iteration 600, loss = 1.4456
Checking accuracy on validation set
Got 557 / 1000 correct (55.70)

Epoch 0, Iteration 700, loss = 1.2586
Checking accuracy on validation set
Got 535 / 1000 correct (53.50)

Epoch 1, Iteration 0, loss = 1.0527
Checking accuracy on validation set
Got 567 / 1000 correct (56.70)

Epoch 1, Iteration 100, loss = 1.1729
Checking acc

## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.

In [30]:
AttentionResnet = model
check_accuracy(loader_test, AttentionResnet)

Checking accuracy on test set
Got 7582 / 10000 correct (75.82)


75.82

## Inline Question 3: Rank the above models based on their performance on test dataset (15 points)
( You are encouraged to run each of the experiments (training) at
least 3 times to get an average estimate )

Report the test accuracies alongside the model names. For example, 1. Vanilla CNN (57.45%, 57.99%).. etc

1. <br /> Attention Resnet10 (76.08,75.39,75.82) Avg: 75.76
2. <br /> Vanilla Resnet10 (74.95,76.03,75.31) Avg: 75.43
3. <br /> Double Attention CNN (61.48,61.81,61.43) Avg: 61.57
4. <br /> Early Attention CNN (61.35,61.15,59.39) Avg: 60.63
5. <br /> Late Attention CNN (59.8,58.64,60.01) Avg: 59.48
6. <br /> Vanilla CNN (56.54,56.16,57.22) Avg: 56.64

### Bonus Question (Ungraded): Can you give a possible explanation that supports the rankings?
Your Answer: Attention Resnet and Vanilla resnet doesn't have much difference in performance. This is because Resnet itself is a complex model to campure enough information, attention doesn't paly any important role. However, Compared to Vanilla CNN, all attention CNN's performs better due to extra complexity in the model and the model able to emphasise more on features learned. Its Double attension performs better due to presense of attension both on the raw data side and the feature side. Early Attention is not expected to perform better than late attention as late attention learns more on the deep feature than on the shallow feature early attention learns. However it is observed Early attention performs better than Late attention.